In [1]:
# Import Necessary Libraries
import open3d as o3d
import numpy as np
from pc_skeletor.laplacian import SLBC
from scipy.spatial import cKDTree
import time
import laspy
import os
import copy
from math import ceil
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from math import ceil
from sklearn.cluster import KMeans
import vtk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Point Cloud Registration

In [2]:
# compute FPFH (Fast Point Feature Histograms)
def compute_fpfh(pcd):
    radius_normal = 0.4
    # print(f":: Estimating normals with search radius {radius_normal:.3f}.")
    pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = 0.4
    # print(f":: Computing FPFH feature with search radius {radius_feature:.3f}.")
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=30)
    )
    return pcd_fpfh

# perform Fast Global Registration (FGR) between source and target point clouds
def perform_fast_global_registration(source, target, source_fpfh, target_fpfh):
    distance_threshold = 0.4
    # print(f":: Applying fast global registration with distance threshold {distance_threshold:.3f}")
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold)
    )
    return result

# save the registration result by transforming
def save_registration(source, target, transformation):
    # Compute FPFH features for source and target point clouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    return source_temp

# perform multiple iterations of registration and choose the best result
def iterative_registration(source_ply, target_ply, iterations):
    source_fpfh = compute_fpfh(source_ply)
    target_fpfh = compute_fpfh(target_ply)
    best_rmse = float('inf')
    best_result = None

    # Iterate the registration process
    for _ in range(iterations):
        result = perform_fast_global_registration(source_ply, target_ply, source_fpfh, target_fpfh)
        if result.inlier_rmse < best_rmse:
            best_rmse = result.inlier_rmse
            best_result = result
            if best_rmse < 0.01:
                break
    return best_result


In [3]:
# Visualize Registration Results
def draw_registration_result(point_cloud, window_name):
    visualizer = o3d.visualization.Visualizer()
    visualizer.create_window(window_name=window_name)
    
    render_option = visualizer.get_render_option()
    render_option.background_color = [0, 0, 0]
    render_option.light_on = False
    visualizer.add_geometry(point_cloud)
    
    visualizer.run()
    visualizer.destroy_window()

In [4]:
## Lidar Point Cloud and UAV Dormancy Period Point Cloud Registration

In [5]:
# Define file paths and directories
lidar_pcd_folder = r"E:\Pear\Lidar\Dormancy Stage\Result_dir\pcd"
uav_dormant_pcd_folder = r"E:\Pear\UAV\Dormancy Stage\Result_dir\pcd"
uav_flower_pcd_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\pcd"

base_dir = r"E:\Pear"
lidar_mesh_dir = os.path.join(base_dir, "Lidar", "Dormancy Stage", "Result_dir", "skeleton", "mesh")
uav_dormant_mesh_dir = os.path.join(base_dir, "UAV", "Dormancy Stage", "Result_dir", "skeleton", "mesh")

# Iterate over the meshes
if len(os.listdir(lidar_mesh_dir)) == len(os.listdir(uav_dormant_mesh_dir)):
    model_names = os.listdir(lidar_mesh_dir)
    for model_name in model_names:
        lidar_mesh_path = os.path.join(lidar_mesh_dir, model_name, "02_skeleton_SLBC.ply")
        uav_dormant_mesh_path = os.path.join(uav_dormant_mesh_dir, model_name, "02_skeleton_SLBC.ply")

        source_ply = o3d.io.read_point_cloud(uav_dormant_mesh_path)
        target_ply = o3d.io.read_point_cloud(lidar_mesh_path)
        
        lidar_pcd_path = os.path.join(lidar_pcd_folder, model_name+".pcd")
        uav_dormant_pcd_path = os.path.join(uav_dormant_pcd_folder, model_name+".pcd")
        
        source_pcd = o3d.io.read_point_cloud(uav_dormant_pcd_path)
        target_pcd = o3d.io.read_point_cloud(lidar_pcd_path)

        best_registration = iterative_registration(source_ply, target_ply, 100)

        icp_result = o3d.pipelines.registration.registration_icp(
            source_ply, target_ply, 100, best_registration.transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=500)
        )

        transformed_pcd = save_registration(source_pcd, target_pcd, icp_result.transformation)

        # Save the transformed point cloud
        save_dir = os.path.join(base_dir, "UAV", "Dormancy Stage", "Result_dir", "Transformed_PCDs")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"transformed_{model_name}.pcd")
        o3d.io.write_point_cloud(save_path, transformed_pcd)
        print(f"Transformed point cloud saved to {save_path}")

Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a1.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a2.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a3.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a4.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a5.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a6.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a7.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_a8.pcd
Transformed point cloud saved to E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs\transformed_b1.pcd
Transformed point cloud saved to E:\P

In [6]:
### UAV Dormancy Period and UAV Flowering Period Point Cloud Registration

In [8]:
# Define file paths and directories
base_dir = r"E:\Pear"
uav_flower_mesh_dir = os.path.join(base_dir, "UAV", "Flowering Stage", "Result_dir", "skeleton", "mesh")
uav_dormant_mesh_dir = os.path.join(base_dir, "UAV", "Dormancy Stage", "Result_dir", "skeleton", "mesh")

# Iterate over the meshes
if len(os.listdir(uav_flower_mesh_dir)) == len(os.listdir(uav_dormant_mesh_dir)):
    model_names = os.listdir(uav_flower_mesh_dir)
    for model_name in model_names:
        uav_flower_mesh_path = os.path.join(uav_flower_mesh_dir, model_name, "02_skeleton_SLBC.ply")
        uav_dormant_mesh_path = os.path.join(uav_dormant_mesh_dir, model_name, "02_skeleton_SLBC.ply")

        source_ply = o3d.io.read_point_cloud(uav_dormant_mesh_path)
        target_ply = o3d.io.read_point_cloud(uav_flower_mesh_path)
        
        uav_flower_pcd_path = os.path.join(uav_flower_pcd_folder, model_name+".pcd")
        uav_dormant_pcd_path = os.path.join(uav_dormant_pcd_folder, model_name+".pcd")
        
        source_pcd = o3d.io.read_point_cloud(uav_dormant_pcd_path)
        target_pcd = o3d.io.read_point_cloud(uav_flower_pcd_path)

        best_registration = iterative_registration(source_ply, target_ply, 100)

        icp_result = o3d.pipelines.registration.registration_icp(
            source_ply, target_ply, 100, best_registration.transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=500)
        )

        transformed_pcd = save_registration(source_pcd, target_pcd, icp_result.transformation)

        # Save the transformed point cloud
        save_dir = os.path.join(base_dir, "UAV", "Flowering Stage", "Result_dir", "Transformed_PCDs")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"transformed_{model_name}.pcd")
        o3d.io.write_point_cloud(save_path, transformed_pcd)
        print(f"Transformed point cloud saved to {save_path}")

Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a1.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a2.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a3.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a4.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a5.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a6.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a7.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_a8.pcd
Transformed point cloud saved to E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs\transformed_b1.pcd
Transformed point cloud save

# Flower Cluster Extraction

In [9]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [10]:
# Nearest neighbor search removes the stem point cloud
def remove_nearest_points(source_point_cloud, target_point_cloud, distance_threshold=0.02):
    # Create a KDTree
    kdtree_target = o3d.geometry.KDTreeFlann(target_point_cloud)
    
    correspondences = []
    corresponding_colors = []
    # # Iterate over each point in the source point cloud
    for idx in range(len(source_point_cloud.points)):
        point_source = np.asarray(source_point_cloud.points[idx])
        # Perform radius search in the KDTree to find points within the distance threshold
        [_, idx_b, _] = kdtree_target.search_radius_vector_3d(point_source, distance_threshold)
        correspondences.append(idx_b)
        corresponding_colors.append(np.asarray(target_point_cloud.colors)[np.asarray(idx_b).flatten()])
        
    correspondences = np.hstack(correspondences)
    correspondences = np.array(correspondences).flatten()
    
    # Find the complement of the indices in the target point cloud
    all_indices = set(np.arange(len(target_point_cloud.points)))
    subset_indices = set(correspondences)
    complement_indices = all_indices - subset_indices
    complement_list = list(complement_indices)

    # Select the complement indices to create the filtered target point cloud
    filtered_target_point_cloud = target_point_cloud.select_by_index(complement_list)
    
    return filtered_target_point_cloud

In [11]:
# Color threshold removes stem point clouds
def remove_by_color(point_cloud, color_threshold=0.8):
    colors = np.asarray(point_cloud.colors)
    # Convert RGB colors to grayscale
    gray_colors = []
    for color in colors:
        gray = color[0]*0.299 + color[1]*0.587 + color[2]*0.114
        gray_colors.append([gray, gray, gray])
    
    # Create a new point cloud with grayscale colors
    graypcd = o3d.geometry.PointCloud()
    graypcd.points = o3d.utility.Vector3dVector(point_cloud.points)
    graypcd.colors = o3d.utility.Vector3dVector(gray_colors)
    
    color_array = np.asarray(graypcd.colors)
    selected_indices = np.where(color_array > color_threshold)[0]
    selected_points = graypcd.select_by_index(selected_indices)
    return selected_points

# Function to perform DBSCAN clustering on a point cloud
def dbscan_cluster(point_cloud, output_folder):
    # Perform DBSCAN clustering on the point cloud
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(point_cloud.cluster_dbscan(eps=0.02, min_points=20, print_progress=False))
    max_label = labels.max()
    
    j = 1
    for i in range(max_label + 1):
        indices = np.where(labels == i)[0]
        # Save the cluster to a file if it contains at least 50 points
        if len(indices) >= 50:
            cluster_point_cloud = point_cloud.select_by_index(indices)
            file_name = f"Dbscan_cluster{j}.pcd"
            file_path = os.path.join(output_folder, file_name)
            o3d.io.write_point_cloud(file_path, cluster_point_cloud)
            j += 1
            
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return point_cloud

In [12]:
# convert a point cloud to a mesh
def point_cloud_to_mesh(point_cloud_path, alpha=1):
    point_cloud = o3d.io.read_point_cloud(point_cloud_path)
    # Create a mesh from the point cloud using the Alpha Shape algorithm
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, alpha)
    mesh.compute_vertex_normals()
    # mesh_area = mesh.get_surface_area()
    return mesh

# compute the surface area and volume of a mesh
def compute_area_volume(mesh_path):
    # Create a VTK reader for PLY files
    vtkReader = vtk.vtkPLYReader()
    vtkReader.SetFileName(mesh_path)
    vtkReader.Update()
    # Get the output polydata from the reader
    polydata = vtkReader.GetOutput()
    mass_properties = vtk.vtkMassProperties()
    mass_properties.SetInputData(polydata)
    
    return mass_properties

# classify data using K-means clustering
def kmeans_classify(trait_data, n_clusters=2):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    kmeans.fit(trait_data)

    kmeans_labels = kmeans.labels_
    # cluster_centers = kmeans.cluster_centers_
    return kmeans_labels

In [13]:
# color a point cloud based on the given label
def color_points_by_label(point_cloud, label):
    if label == 1:
        color = [1.0, 0.0, 0.0]  # Red
    elif label == 0:
        color = [0.0, 1.0, 0.0]  # Green
    else:
        color = [0.5, 0.5, 0.5]  # Gray (for unknown labels)

    point_cloud.paint_uniform_color(color)
    return point_cloud

# fix labels
def fix_label(labels):
    num_zeros = np.sum(labels == 0)
    num_ones = len(labels) - num_zeros

    # label #0 alaways greater than label #1
    if num_zeros > num_ones:
        new_labels = labels
    else:
        new_labels = 1 - labels
        
    return new_labels

# compute the number of flowers given points for single and multi-flower clusters
def compute_cluster_num(single_flower_points, multi_flower_points):
    # Calculate the average number of points using the top 10 largest single flower clusters
    average_point_num = sum(sorted(single_flower_points, reverse=True)[:10]) / 10
    flower_num = [ceil(x / average_point_num) for x in multi_flower_points]
    # Calculate the total number of flowers by summing the estimated counts of flowers in multi-flower clusters
    total_flower_num = sum(flower_num) + len(single_flower_points)
    
    return total_flower_num, average_point_num

In [14]:
# Extract the traits of each tree
def process_tree_traits(tree_result_folder):

    dbscan_folder = os.path.join(tree_result_folder, "dbscan_result")
    mesh_folder = os.path.join(tree_result_folder, "dbscan_mesh")
    tree_trait_list = []

    # Iterate over each tree in the DBSCAN results folder
    for tree_num in os.listdir(dbscan_folder):
        tree_path = os.path.join(dbscan_folder, tree_num)
        single_tree_trait = []
        for name in os.listdir(tree_path):
            cluster_path = os.path.join(tree_path, name)
            pcd = o3d.io.read_point_cloud(cluster_path)
            tree_mesh_folder = os.path.join(mesh_folder, tree_num)
            create_folder_if_not_exists(tree_mesh_folder)
            mesh_path = os.path.join(tree_mesh_folder, name[:-4] + ".ply")
            # Convert the point cloud to a mesh
            mesh = point_cloud_to_mesh(cluster_path)
            o3d.io.write_triangle_mesh(mesh_path, mesh)

            # Compute the mass properties (surface area and volume) of the mesh
            mass_properties = compute_area_volume(mesh_path)
            single_tree_trait.append([len(pcd.points), mass_properties.GetSurfaceArea(), mass_properties.GetVolume()])
        tree_trait_list.append(single_tree_trait)

    return tree_trait_list

In [15]:
# classify and count flowers
def classify_and_count_flowers(tree_result_folder):

    # Process tree traits to get a list of traits for each tree
    tree_trait_list = process_tree_traits(tree_result_folder)
    
    dbscan_folder = os.path.join(tree_result_folder, "dbscan_result")
    tree_num_folders = [os.path.join(dbscan_folder, num) for num in os.listdir(dbscan_folder)]
    all_tree_pcd_paths = []
    for i in range(len(tree_num_folders)):
        pcd_path_list = []
        for name in os.listdir(tree_num_folders[i]):
            cluster_path = os.path.join(tree_num_folders[i], name)
            pcd_path_list.append(cluster_path)
        all_tree_pcd_paths.append(pcd_path_list)

    tree_names = os.listdir(dbscan_folder)
    colored_pcd_output_folder = os.path.join(tree_result_folder, "pcd_cluster_with_color")
    create_folder_if_not_exists(colored_pcd_output_folder)
    flower_num_dict = {}

    # Iterate over the point cloud paths and tree traits for each tree
    for i in range(len(all_tree_pcd_paths)):
        tree_trait = np.array(tree_trait_list[i])
        pcd_path_list = all_tree_pcd_paths[i]
        
        # Classify the tree traits using K-Means
        labels = kmeans_classify(np.asarray(tree_trait_list[i]))
        fixed_labels = fix_label(labels)

        single_flower_points = []
        multi_flower_points = []
        combined_pcd = o3d.geometry.PointCloud()

        for pcd_path, label in zip(pcd_path_list, fixed_labels):
            pcd = o3d.io.read_point_cloud(pcd_path)
            # Color the points in the point cloud based on their label
            colored_pcd = color_points_by_label(pcd, label)
            combined_pcd += colored_pcd
            if label == 0:
                single_flower_points.append(len(pcd.points))
            elif label == 1:
                multi_flower_points.append(len(pcd.points))

        # Compute the number of flower clusters
        cluster_num, average_point_num = compute_cluster_num(single_flower_points, multi_flower_points)
        flower_num_dict[tree_names[i]] = cluster_num

        output_path = os.path.join(colored_pcd_output_folder, tree_names[i] + ".pcd")
        o3d.io.write_point_cloud(output_path, combined_pcd)

In [16]:
def process_tree_data(source_pcd_folder, target_pcd_folder, tree_result_folder):

    source_pcd_names = [name for name in os.listdir(source_pcd_folder)]
    target_pcd_names = [name for name in os.listdir(target_pcd_folder)]

    for i in range(len(source_pcd_names)):
        source_pcd_path = os.path.join(source_pcd_folder, source_pcd_names[i])
        target_pcd_path = os.path.join(target_pcd_folder, target_pcd_names[i])

        source_pcd = o3d.io.read_point_cloud(source_pcd_path)
        target_pcd = o3d.io.read_point_cloud(target_pcd_path)

        # Filter the source point cloud by removing points that are nearest to the target point cloud
        filtered_pcd_by_morphology = remove_nearest_points(source_pcd, target_pcd, distance_threshold=0.05)
        # Further filter the point cloud by color
        filtered_pcd_by_color = remove_by_color(filtered_pcd_by_morphology, color_threshold=0.6)

        db_output_folder = os.path.join(tree_result_folder, "dbscan_result", target_pcd_names[i][:-4])
        create_folder_if_not_exists(db_output_folder)
        
        # Perform DBSCAN clustering on the filtered point cloud and save the results
        clustered_pcd = dbscan_cluster(filtered_pcd_by_color, db_output_folder)

    process_tree_traits(tree_result_folder)
    classify_and_count_flowers(tree_result_folder)

In [17]:
# Original PCD folders

lidar_pcd_folder = r"E:\Pear\Lidar\Dormancy Stage\Result_dir\pcd"
uav_dormant_pcd_folder = r"E:\Pear\UAV\Dormancy Stage\Result_dir\pcd"
uav_flower_pcd_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\pcd"

# PCD folders after rotation and translation
# uav dormant and lidar dormant
transformed_uav_lidar_dormant_pcd_folder = r"E:\Pear\UAV\Dormancy Stage\Result_dir\Transformed_PCDs"
# uav dormant and uav flower
transformed_uav_dormant_flower_pcd_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\Transformed_PCDs"
# Folder for results after extracting flower clusters
flower_clusters_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\flower_clusters"

In [18]:
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
process_tree_data(transformed_uav_dormant_flower_pcd_folder, uav_flower_pcd_folder, flower_clusters_folder)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 109
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 103
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 101
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 102
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 181
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 196
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Op

# Add offset

In [19]:
def add_offset_to_point_clouds(main_folder):
    # Define folder paths
    las_folder = os.path.join(main_folder, "origin_single_tree_las")
    pcd_folder = os.path.join(main_folder, "pcd")
    skeleton_folder = os.path.join(main_folder, "skeleton", "mesh")
    color_flower_folder = os.path.join(main_folder, "flower_clusters", "pcd_cluster_with_color")
    offset_folder = os.path.join(main_folder, "offset")

    # Create offset folders
    pcd_offset_folder = os.path.join(offset_folder, "pcd_with_offset")
    ply_offset_folder = os.path.join(offset_folder, "ply_with_offset")
    color_flower_offset_folder = os.path.join(offset_folder, "color_flower_with_offset")
    os.makedirs(pcd_offset_folder, exist_ok=True)
    os.makedirs(ply_offset_folder, exist_ok=True)
    os.makedirs(color_flower_offset_folder, exist_ok=True)

    # Get list of files
    names = [name[:-4] for name in os.listdir(pcd_folder)]
    las_list = [os.path.join(las_folder, name + ".las") for name in names]
    pcd_list = [os.path.join(pcd_folder, name + ".pcd") for name in names]
    ply_list = [os.path.join(skeleton_folder, name, "02_skeleton_SLBC.ply") for name in names]
    color_flower_list = [os.path.join(color_flower_folder, name + ".pcd") for name in names]

    # Add offsets to point clouds in dbscan_mesh and dbscan_result folders
    dbscan_mesh_folder = os.path.join(main_folder, "flower_clusters", "dbscan_mesh")
    dbscan_result_folder = os.path.join(main_folder, "flower_clusters", "dbscan_result")

    dbscan_mesh_offset_folder = os.path.join(offset_folder, "dbscan_mesh_with_offset")
    dbscan_result_offset_folder = os.path.join(offset_folder, "dbscan_result_with_offset")
    os.makedirs(dbscan_mesh_offset_folder, exist_ok=True)
    os.makedirs(dbscan_result_offset_folder, exist_ok=True)

    for i in range(len(names)):
        las = laspy.read(las_list[i])
        pcd = o3d.io.read_point_cloud(pcd_list[i])
        ply = o3d.io.read_point_cloud(ply_list[i])
        color_flower = o3d.io.read_point_cloud(color_flower_list[i])

        min_x, min_y, min_z = np.min(las.x), np.min(las.y), np.min(las.z)

        # Shift point cloud data
        pcd_points = np.asarray(pcd.points)
        pcd_points_shifted = pcd_points + np.array([min_x, min_y, min_z])

        ply_points = np.asarray(ply.points)
        ply_points_shifted = ply_points + np.array([min_x, min_y, min_z])

        color_flower_points = np.asarray(color_flower.points)
        color_flower_points_shifted = color_flower_points + np.array([min_x, min_y, min_z])

        # Update shifted point cloud data
        pcd.points = o3d.utility.Vector3dVector(pcd_points_shifted)
        ply.points = o3d.utility.Vector3dVector(ply_points_shifted)
        color_flower.points = o3d.utility.Vector3dVector(color_flower_points_shifted)

        # Save point cloud files with added offsets
        pcd_offset_path = os.path.join(pcd_offset_folder, names[i] + ".ply")
        ply_offset_path = os.path.join(ply_offset_folder, names[i] + ".ply")
        color_flower_offset_path = os.path.join(color_flower_offset_folder, names[i] + ".ply")

        o3d.io.write_point_cloud(pcd_offset_path, pcd)
        o3d.io.write_point_cloud(ply_offset_path, ply)
        o3d.io.write_point_cloud(color_flower_offset_path, color_flower)

        mesh_folder = os.path.join(dbscan_mesh_folder, names[i])
        result_folder = os.path.join(dbscan_result_folder, names[i])

        mesh_offset_folder = os.path.join(dbscan_mesh_offset_folder, names[i])
        result_offset_folder = os.path.join(dbscan_result_offset_folder, names[i])
        os.makedirs(mesh_offset_folder, exist_ok=True)
        os.makedirs(result_offset_folder, exist_ok=True)

        for mesh_file in os.listdir(mesh_folder):
            mesh_path = os.path.join(mesh_folder, mesh_file)
            mesh = o3d.io.read_triangle_mesh(mesh_path)

            mesh_vertices = np.asarray(mesh.vertices)
            mesh_vertices_shifted = mesh_vertices + np.array([min_x, min_y,min_z])
            mesh.vertices = o3d.utility.Vector3dVector(mesh_vertices_shifted)

            mesh_offset_path = os.path.join(mesh_offset_folder, mesh_file)
            o3d.io.write_triangle_mesh(mesh_offset_path, mesh)

        for result_file in os.listdir(result_folder):
            result_path = os.path.join(result_folder, result_file)
            result = o3d.io.read_point_cloud(result_path)

            result_points = np.asarray(result.points)
            result_points_shifted = result_points + np.array([min_x, min_y, min_z])
            result.points = o3d.utility.Vector3dVector(result_points_shifted)

            result_offset_path = os.path.join(result_offset_folder, result_file[:-4]+".ply")
            o3d.io.write_point_cloud(result_offset_path, result)

In [20]:
main_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir"
add_offset_to_point_clouds(main_folder)